# Stroj na předpověď

Náš první program, '<b>Simple Prediction Machine</b>' je velmi zjednodušenou ukázkou principů Machine Learningu. Pattern, který si zde ukážeme, je však aplikovaný při učení většiny supervised ML modelů. 

![alt text](pictures/machine_learning_small.jpg  "Machine Learning")

Ještě než začneme - v ML je zažité označovat vstup jako X, požadovaný výstup Y a předpověď modelu jako ŷ [y-hat]. Náš program bude mít jeden vstup a jeden výstup. Vstupem je zadání a výstupem je řešení. Stejně jako na obrázku.

In [1]:
import tensorflow as tf

nr_inputs = 1

X a Y reprezentují naše tréningová data. Úkolem SPM je najít správný vztah mezi vstupem a výstupem s použitím funkcí Tensorflow. Už víte, jaký je v datech reprezentován vztah?

In [2]:
X =   [ [8.],       [45.],      [323.],     [0.0], [100] ]
Y =   [ [4.970970], [27.96170], [200.7029], [0.0], [62.13712] ]

picture of speedometer

<b>Tréningová data, která máme k dispozici, rozdělíme.</b> Část z nich použijeme na učení našeho modelu, druhou si necháme k tomu, abychom si ověřili, zda model je schopen provádět predikci i nad daty, která nebyla použita při tréningu.

In [3]:
train_x =   X[:3]
train_y =   Y[:3]
print('train_x: ', train_x)
print('train_y: ', train_y)

test_x  =   X[3:]
test_y  =   Y[3:]
print('test_x: ', test_x)
print('test_y: ', test_y)

train_x:  [[8.0], [45.0], [323.0]]
train_y:  [[4.97097], [27.9617], [200.7029]]
test_x:  [[0.0], [100]]
test_y:  [[0.0], [62.13712]]


V programu definujeme ve scope 'input' placeholder input a ve scope 'target' placeholder, kam budeme ukládat požadovaný výsledek k příslušnému vstupu. Definice scope není povinná, má zde jen pomoci lepší přehlednosti grafu zobrazeného na Tensorboardu.

In [4]:
with tf.name_scope('input'):
   x = tf.placeholder(tf.float32, [None, nr_inputs], name='input')

with tf.name_scope('target'):
   y = tf.placeholder(tf.float32, [None, 1], name='target')

Zde je vnitřek naší 'prediction machine'. Za fasádou strojového učení se v našem zjednodušeném příkladu skrývá lineární funkce x * W. Jak uvidíte později, každý jeden ML neuron, má v sobě též schovanou obyčejnou lineární funkci. Ještě se vrátíme k Tensoru Variable, který je pro Tensorflow takzvaný Trainable Tensor.  Trainable Tensory mohou být měněny funkcemi Tensorflow tak, aby funkce konvergovala ke správnému výsledku. 

![alt text](pictures/ML_pattern.png "Suervised ML Pattern")

In [5]:
with tf.name_scope('p_machine'):

    with tf.name_scope('Wtrainable'):
        # Weight - Variable is by default trainable
        W = tf.Variable(tf.random_uniform([nr_inputs, 1], -1, 1), name="rand_W")
        tf.trainable_variables()
    
    with tf.name_scope('prediction_fn'):
        pred = tf.matmul(x, W, name='Wx')

Jak se Tensorflow přilíží ke správnému výsledku? Tensorflow provede úpravu hodnoty tensoru W a srovnáním se správným výsledkem (pred - y) spočítá, zda se hodnota chyby zvětšila, nebo zmenšila. Potom se vydá v optimalizaci tensoru W takovým směrem, aby hodnota chyby po každé operaci klesla. Tento přístup se nazývá <b>Gradient Descent</b>. Parametry Gradient Descent optimalizace, např. hodnota kroku, se dají vložit jako parametr funkce. Pokud bychom parametr slope nastavili příliš velký, pravděpodoně se nepovede najít optimální hodnotu. Pokud však bude příliš malý, model bude ke správnému výsledku konvergovat déle.

In [26]:
with tf.name_scope("training") as scope:
    
    with tf.name_scope("cost_fn") as scope:
        cost_function = tf.reduce_sum(pred - y) ** 2
        
    #train = tf.train.AdamOptimizer().minimize(cost_function)
    #vyzkoušej  
    #train = tf.train.GradientDescentOptimizer(0.01).minimize(cost_function)
    #výpočet by měl selhat, hodnota NaN, příliš velký slope 
    train = tf.train.GradientDescentOptimizer(0.000001).minimize(cost_function)

![alt text](pictures/gradient_descent_slope.png "Gradient Descent slope")

Možná trochu matoucí názvy <b>'cost'</b>, případně <b>'loss'</b> jsou v ML zažité termíny pro <b>chybu</b>.

In [28]:
with tf.Session() as sess:
    writer = tf.summary.FileWriter("./tensorboard_example", sess.graph)

    tf.summary.scalar("cost", cost_function)
    merged_summary = tf.summary.merge_all()

Tady už vše necháme na Tensorflow, až bude náš 'prediction machine' model generovat dostatečně malou chybu, proces modifikace proměnné Variable ukončíme.Podíváme se pak do Tensorboardu a celý program si tam ještě jednou rozebereme. 

In [25]:
    sess.run(tf.global_variables_initializer())

    cost_function_result = 1000.0
    training_epoch = 0
    
    while cost_function_result > 0.01:
        training_epoch += 1
        _, cost_function_result = sess.run([train, cost_function], feed_dict={x: train_x, y: train_y})
        if training_epoch % 1000 == 0:
            writer.add_summary(
                sess.run(merged_summary, feed_dict={x: train_x, y: train_y}), training_epoch)
            print('epoch: ', training_epoch, ' cost: ', cost_function_result)
    
    # feed predictor with test data and print results
    res = sess.run(pred, feed_dict={x: test_x})
    print("Weight(s): ")
    print(sess.run(W))
    print("Predidctions for: ")
    print(test_x)
    print("is: ")
    print(res)
    print('100Km/h = 62.13712mph')

Weight(s): 
[[ 0.62154001]]
Predidctions for: 
[[0.0], [100]]
is: 
[[  0.        ]
 [ 62.15399933]]
100Km/h = 62.13712mph


# Odkazy:

https://iamtrask.github.io/2015/07/27/python-network-part2/